# Corporate green bonds

Source: https://doi.org/10.1016/j.jfineco.2021.01.010

## 1. Introduction

The study analyzes China’s corporate green bond market by merging issuance and stock data, applying descriptive statistics and event study regressions.

While the market has seen rapid green bond growth since 2016, accounted mainly in the industrial and utilities sectors, the regression results for various CARs show no significant impact, suggesting that stock price reactions to announcements are not driven by unrelated trends around the event date.

The findings highlight that, despite a positive policy environment and initial issuance enthusiasm, the real impact of green bond announcements on stock prices remains limited and statistically insignificant across most windows.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

## 2. Preprocessing and descriptive statistics

In [ ]:
# Corporate and Company Bond Data
corpdata = pd.read_csv("EDE20241224.csv")
corpdata1 = pd.read_csv("firm bond.csv")
corpdata.rename({"债务主体中文名称↑":"债务主体中文名称"},axis=1,inplace=True)
corpdata1.rename({"是否ESG(绿色)债券↓":"是否ESG(绿色)债券"},axis=1,inplace=True)
corpdata = corpdata[list(corpdata1.columns)]
corpdata = pd.concat([corpdata,corpdata1],axis=0)
corpdata.columns

In [ ]:
# Annual Distribution and Total Volume of Green Bonds
greendata = corpdata[corpdata['是否ESG(绿色)债券'] == '是'].reset_index(drop=True)
greendata["Year"] = greendata["发行公告日"].apply(lambda x: x.split("-")[0])
sum = greendata.groupby("Year").size().reset_index()
sum = sum.merge(greendata.groupby("Year")["发行总额\r\n[单位] 十亿元"].sum().reset_index(), on="Year")
sum.columns = ["Year","Bonds","Amount-b"]
sum


Findings: From 2016 to 2024, the issuance of green bonds in China showed an overall upward trend in both quantity and amount.

Issuance peaked in 2020 (164 bonds, 152.6 billion yuan), followed by a decline in 2021 (122 bonds, 103.9 billion yuan).

There was significant growth again in 2022-2023, with 2023 reaching the highest issuance (240 bonds, 161.1 billion yuan).

In 2024, the number of issuances dropped to 156 bonds with an amount of 135.0 billion yuan.

The trend reflects increasing policy support and market emphasis on green finance.

In [ ]:
plt.figure(figsize=(8,4))
bar_width = 0.35
plt.bar(sum['Year'].astype(float) - 1/2*bar_width, sum['Bonds'], bar_width, label='Bonds')
plt.bar(sum['Year'].astype(float) + 1/2*bar_width, sum['Amount-b'], bar_width, label='Amount-b')
plt.xlabel('Year')
plt.ylabel('Count / Amount-b')
plt.title('Yearly Bonds and Amount of Bonds counted by billion')
plt.xticks(sum['Year'].astype(float), sum['Year'])
plt.legend()
plt.show()

Findings: 2020 and 2022-2023 were peak issuance periods, reflecting policy promotion and growing market demand.

Both quantity and amount declined in 2024, potentially indicate market adjustments or policy changes.

In [ ]:
# Industry Distribution and Total Volume of Green Bonds
ind = greendata.groupby("主体行业").size().reset_index()
ind = ind.merge(greendata.groupby("主体行业")["发行总额\r\n[单位] 十亿元"].sum().reset_index(),
    on="主体行业").sort_values("发行总额\r\n[单位] 十亿元",ascending=False)
ind.columns = ["Industry","Bonds","Amount-b"]
ind

Findings: Results indicate that green bonds are more concentrated in environmentally related industries (industrial, utilities, energy).

Other industries such as energy, real estate, consumer discretionary, healthcare, consumer staples, and information technology have relatively lower issuance volumes.

## 3. Feature Engineering

In [ ]:
# Corporate Stock Return Table
stockdata = pd.read_csv("绿色债日股收益.CSV",encoding='gbk',index_col=0)
stockdate = stockdata.iloc[2:,0].reset_index(drop=True)
stockdata = stockdata.drop("Unnamed: 85",axis=1)
stockdata

In [ ]:
stockname = stockdata.iloc[0,0:42].reset_index()
stockname.columns = ["code","name"]
stockname.head()

In [ ]:
stockexclose = stockdata.iloc[2:,0:42]
columns = list(stockexclose.columns)
stockexclose = stockexclose.stack().reset_index()
stockexclose.columns = ["date","stkcode","exclose"]
stockexclose

In [ ]:
stockclose = stockdata.iloc[2:,42:]
stockclose.columns = columns
stockclose = stockclose.stack().reset_index()
stockclose.columns = ["date","stkcode","close"]
stockclose

In [ ]:
stockret = pd.merge(stockclose,stockexclose,on=["date","stkcode"],how = "inner")
stockret["close"] = stockret["close"].astype(float)
stockret["exclose"] = stockret["exclose"].astype(float)
stockret["ret"] = (stockret["close"] - stockret["exclose"])/stockret["exclose"]
stockret["date"] = pd.to_datetime(stockret["date"],format="%Y%m%d")
stockret.sort_values(["stkcode","date"],ascending=True).reset_index(drop=True,inplace=True)
stockret

In [ ]:
stkissue = corpdata.loc[(corpdata['是否ESG(绿色)债券'] == '是') & (corpdata["是否上市公司"] == 1),["债务主体中文名称","发行公告日"]].reset_index(drop=True)
stkissue = stkissue.groupby("债务主体中文名称").min()

In [ ]:
stockname["fullname"] = ['武汉三镇实业控股股份有限公司', '深圳能源集团股份有限公司', '深圳高速公路集团股份有限公司', '华电国际电力股份有限公司',
                         '江苏宁沪高速公路股份有限公司', '中国长江电力股份有限公司', '上海临港控股股份有限公司', '重庆三峰环境集团股份有限公司',
                         '中国电力建设股份有限公司', '平顶山天安煤业股份有限公司', '广州港股份有限公司', '中节能风力发电股份有限公司',
                         '宁波能源集团股份有限公司', '上海张江高科技园区开发股份有限公司', '华新水泥股份有限公司', '浙江钱江生物化学股份有限公司',
                         '厦门国贸集团股份有限公司', '申能股份有限公司', '中化国际(控股)股份有限公司', '上海浦东建设股份有限公司',
                         '国电南京自动化股份有限公司', '陕西建工集团股份有限公司', '广西桂冠电力股份有限公司', '厦门建发股份有限公司',
                         '特变电工股份有限公司', '厦门象屿股份有限公司', '宝山钢铁股份有限公司', '内蒙古包钢钢联股份有限公司', '中国能源建设股份有限公司',
                         '中国建材股份有限公司', '山东美晨科技股份有限公司', '中国铁建股份有限公司', '中国水发兴业能源集团有限公司',
                         '新疆北新路桥集团股份有限公司', '新疆中泰化学股份有限公司', '中材科技股份有限公司', '吉林电力股份有限公司',
                         '启迪环境科技发展股份有限公司', '天津泰达股份有限公司', '中节能太阳能股份有限公司', '天津中绿电投资股份有限公司',
                         '四川省新能源动力股份有限公司']
stockname = stockname.merge(stkissue,left_on="fullname",right_on="债务主体中文名称",how="inner")
stockname["发行公告日"] = pd.to_datetime(stockname["发行公告日"],format="%Y-%m-%d")
stockname = stockname.drop([6,29,32],axis=0).reset_index(drop=True)
# Retain A-shares and ChiNext (Growth Enterprise Market) stocks from the Shanghai and Shenzhen exchanges
stockname.shape

In [ ]:
# Add Market Risk Factors
mktfactor = pd.read_csv("STK_MKT_THRFACDAY.csv")
mktfactor = mktfactor[mktfactor["MarkettypeID"] == "P9709"].reset_index(drop=True)
mktfactor["TradingDate"] = pd.to_datetime(mktfactor["TradingDate"],format="%Y-%m-%d")
mktfactor

## 4. Window Setup

In [ ]:
# Add Corresponding Stock Dates to Create Event Windows
stockret["time"] = stockret.groupby("stkcode")["date"].rank(ascending=True)
stockret = stockret.sort_values(["stkcode","date"],ascending=True).reset_index(drop=True)
stockret

In [ ]:
# Define an Event Window Function for Estimation and Easy Reference
def get_window_stkret(pre_window,post_window):
    stkret = pd.DataFrame()
    for i in stockname["code"]:
        issuedate = stockname.loc[stockname["code"] == i,"发行公告日"].values[0]
        issuetime = stockret.loc[((stockret["date"] == issuedate) & (stockret["stkcode"] == i)) ,"time"].values[0]
        stockname.loc[stockname["code"] == i,"time"] = issuetime
        pre_issue = issuetime + pre_window
        post_issue = issuetime + post_window
        stkret1 = stockret[(stockret["stkcode"] == i) & (stockret["time"] >= pre_issue) & (stockret["time"] <= post_issue)].reset_index(drop=True)
        stkret1["time"] -= issuetime
        stkret = pd.concat([stkret,stkret1],axis=0)
    stkret.reset_index(drop=True,inplace=True)
    stkret["ret_mkt"] = stkret["date"].map(mktfactor.set_index("TradingDate")["RiskPremium2"])
    return stkret, stockname

In [ ]:
# Test run of function
stkret, stockname = get_window_stkret(-5,5)
stockname.loc[27,"time"] 

It shows that China Energy Engineering Corporation issued green bonds relatively early, so there is a lack of the required 220 trading days' stock return data before issuance.

In [ ]:
stockname = stockname.drop(27,axis=0).reset_index(drop=True)
stockname.shape

## 5. OLS estimation on windows

In [ ]:
# Calculate OLS estimates of market model coefficients for each company
stkret, stockname = get_window_stkret(-220,-21)
stock_regression = stockname.copy()
for i in stockname["code"]:
    stkret1 = stkret[stkret["stkcode"] == i]
    stock_regression.loc[stock_regression["code"] ==i, "const"] = sm.OLS(stkret1["ret"],sm.add_constant(stkret1["ret_mkt"])).fit().params.values[0]
    stock_regression.loc[stock_regression["code"] ==i, "beta"] = sm.OLS(stkret1["ret"],sm.add_constant(stkret1["ret_mkt"])).fit().params.values[1]
    stock_regression.loc[stock_regression["code"] ==i, "var_ar"] = sm.OLS(stkret1["ret"],sm.add_constant(stkret1["ret_mkt"])).fit().mse_resid
stock_regression.head()

In [ ]:
def car_calculation(pre_window,post_window):
    stkret, stockname = get_window_stkret(pre_window,post_window)
    for i in stockname["code"]:
        stkret1 = stkret[stkret["stkcode"] == i]
        stock_regression.loc[stock_regression["code"] ==i, "CAR["+ str(pre_window) +","+ str(post_window)+"]"
                             ] = stkret1["ret"].sum() - stkret1["ret_mkt"].sum()*stock_regression.loc[stock_regression["code"] ==i, "beta"].values[0]
    return stock_regression

In [ ]:
stock_regression = car_calculation(-20,-11)
stock_regression = car_calculation(-10,-6)
stock_regression = car_calculation(-5,10)
stock_regression = car_calculation(11,20)
stock_regression = car_calculation(21,60)
stock_regression.head()

In [ ]:
stock_regression.iloc[:,-5:].describe()

Findings: The regression result of various CAR shows no significant impact. 

This suggests that the results are not driven by unrelated trends around the event date.